In [25]:
import json
import re
import os
import glob
import string
from gensim.models import Word2Vec

from tqdm import tqdm

In [26]:
files=glob.glob(os.path.join('data', 'wikidata', '*', '*'))


In [27]:
texts=[]
for file in files:
    with open(file, 'r') as f:
        for line in f:
            texts.append(json.loads(line)['text'])

print(len(texts))

132818


In [28]:
def replace_strings(texts, replace):
    new_texts=[]
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    bengali_pattern=re.compile('[०-९]+', flags=re.I)
    
    for text in tqdm(texts):
        for r in replace:
            text=text.replace(r[0], r[1])
        text=emoji_pattern.sub(r'', text)
        text=english_pattern.sub(r'', text)
        text=bengali_pattern.sub(r'', text)
        text=re.sub(r'\s+', ' ', text).strip()
        new_texts.append(text)

    return new_texts


In [29]:
replace=[('\u200c', ' '),
         ('\u200b', ' '),
        ('\n', ' '),
        ('\n5', ' '),
        ]

texts=remove_punc(texts)
texts=replace_strings(texts, replace)

100%|██████████| 132818/132818 [00:32<00:00, 4145.36it/s]


In [31]:
body=[article.split('।') for article in texts]
body=[item for sublist in body for item in sublist]
body=[item.strip() for item in body if len(item.split())>1]

body=[item.split() for item in body]

print(body[:10])

[['नैदानिक', 'परीक्षा', 'शिक्षा', 'व्यक्ति', 'में', 'किसी', 'प्रकार', 'की', 'असामान्यता', 'अयोग्यता', 'अक्षमता', 'व्यतिक्रम', 'अथवा', 'व्याधि', 'के', 'स्वरूप', 'की', 'ठीक', 'पहचान', 'अर्थात्', 'निदान', 'डायग्नासिस', 'नैदानिक', 'परीक्षा', 'कहलाती', 'है'], ['यह', 'मनोशारीरिक', 'अव्यवस्थाओं', 'की', 'ज्ञात', 'लक्षणावली', 'के', 'आधार', 'पर', 'करने', 'के', 'लिए', 'विज्ञों', 'द्वारा', 'सुस्थिर', 'की', 'गई', 'जाँच', 'की', 'विशिष्ट', 'प्रणाली', 'है'], ['नैदानिक', 'परीक्षाओं', 'की', 'अब', 'हजारों', 'तरह', 'की', 'प्रश्नावलियाँ', 'एवं', 'पद्धतियाँ', 'चल', 'पड़ी', 'है', 'जिनमें', 'व्यक्ति', 'के', 'मनोजगत्', 'की', 'सूक्ष्मतल', 'विकृति', 'की', 'पहचान', 'प्रायोगिक', 'धरातल', 'पर', 'कर', 'ली', 'जाती', 'है'], ['ये', 'परीक्षाएँ', 'व्यक्तिगत', 'रूप', 'में', 'भी', 'होती', 'हैं', 'और', 'समूहगत', 'भी'], ['आजकल', 'समूह', 'परीक्षाएँ', 'ग्रूप', 'टेस्ट्स', 'शिक्षण', 'संस्थाओं', 'के', 'माध्यम', 'से', 'तथा', 'सैनिक', 'विभाग', 'और', 'कार्यालयीय', 'कर्मचारियों', 'की', 'सहायता', 'से', 'काफी', 'मात्रा', 'में', 'संपन्न

In [32]:
model = Word2Vec(body, size=200, window=5, min_count=10)

In [33]:
print("What are the words most similar to beautiful")
model.wv.most_similar('सुंदर', topn=5)

What are the words most similar to beautiful


[('सुन्दर', 0.8623796701431274),
 ('मनमोहक', 0.7837326526641846),
 ('मनोरम', 0.773344874382019),
 ('आकर्षक', 0.7708183526992798),
 ('खूबसूरत', 0.7240737676620483)]

In [34]:
print("What is king - man + woman =?")
model.wv.most_similar(positive=['राजा', 'महिला'], negative=['आदमी'], topn=5)


What is king - man + woman =?


[('नरेशों', 0.5417574048042297),
 ('नरेश', 0.5336947441101074),
 ('चोल', 0.5314998626708984),
 ('मिथिला', 0.5177757143974304),
 ('महारानी', 0.5176520347595215)]

In [35]:
print("What is Father + Girl - Boy =?")
model.wv.most_similar(positive=['पिता', 'लड़की'], negative=['लड़का'], topn=5)

What is Father + Girl - Boy =?


[('मातापिता', 0.6321265697479248),
 ('पति', 0.6085711717605591),
 ('दादी', 0.5852773785591125),
 ('विधवा', 0.574411928653717),
 ('भतीजी', 0.5679188966751099)]

In [ ]:
print("Find the odd one out")
model.wv.doesnt_match("".split())